In [1]:
from sympy import Matrix, sqrt
from sympy.physics.quantum import TensorProduct
from quantum import controlled_u, inverted_controlled_u, PAULI_X, HADAMARD, PAULI_Z

# Bell's basis
Consider the following quantum circuit on a 2-qubit system:

```
-----*-----|H|-----
     |
----|X|------------
```

In [2]:
bell = TensorProduct(HADAMARD, Matrix.eye(2)) * controlled_u(PAULI_X)
display(bell)

Matrix([
[sqrt(2)/2,         0,          0,  sqrt(2)/2],
[        0, sqrt(2)/2,  sqrt(2)/2,          0],
[sqrt(2)/2,         0,          0, -sqrt(2)/2],
[        0, sqrt(2)/2, -sqrt(2)/2,          0]])

# Super dense coding
Alice has 2 bits. Bob can prepare an entangled 2-qubit system, send one qubit to Alice. After Alice applies some unitary to this qubit, and returns it back to Bob, Bob can recover Alice's 2 bits by measuring the 2-qubit system:

![](./assets/super-dense-coding.jpeg)

In [3]:
psi = Matrix([1/sqrt(2), 0, 0, 1/sqrt(2)])

def super_dense_coding(a: int, b: int):
    bob = psi
    if a:
        bob = TensorProduct(PAULI_X, Matrix.eye(2)) * bob
    if b:
        bob = TensorProduct(PAULI_Z, Matrix.eye(2)) * bob
    bob = TensorProduct(Matrix.eye(2), HADAMARD) * inverted_controlled_u(PAULI_X) * bob
    return bob

for (a, b) in [
    (0, 0), (0, 1), (1, 0), (1, 1)
]:
    display(f"Alice's bits are {a}{b}; Bob's final state is: ", super_dense_coding(a, b))


"Alice's bits are 00; Bob's final state is: "

Matrix([
[1],
[0],
[0],
[0]])

"Alice's bits are 01; Bob's final state is: "

Matrix([
[0],
[1],
[0],
[0]])

"Alice's bits are 10; Bob's final state is: "

Matrix([
[0],
[0],
[1],
[0]])

"Alice's bits are 11; Bob's final state is: "

Matrix([
[ 0],
[ 0],
[ 0],
[-1]])

# Teleportation
![teleportation](./assets/teleportation.png)

In [17]:
from sympy import Symbol

alpha0, alpha1 = Symbol("alpha_0", complex=True), Symbol("alpha_1", complex=True)
alice_qubit = Matrix([alpha0, alpha1])
init_qubit = Matrix([1/sqrt(2), 0, 0, 1/sqrt(2)])

# Step 1: Alice combines her qubit and the first shared qubit into a 2-qubit system
complete_system = TensorProduct(alice_qubit, init_qubit)

# Step 2: Alice applies CNOT and (H x I) to her two qubits
#   in other words, Alice measures her 2 qubits in Bell's basis
complete_system = TensorProduct(TensorProduct(HADAMARD, Matrix.eye(2)), Matrix.eye(2)) * TensorProduct(controlled_u(PAULI_X), Matrix.eye(2)) * complete_system

display(complete_system)

Matrix([
[ alpha_0/2],
[ alpha_1/2],
[ alpha_1/2],
[ alpha_0/2],
[ alpha_0/2],
[-alpha_1/2],
[-alpha_1/2],
[ alpha_0/2]])

The state of the 3-qubit system right before Alice measures her 2 qubits is:

$$
\begin{aligned}
\vert \psi \rangle
&= \frac{\alpha_0}{2} \vert 000 \rangle
+ \frac{\alpha_1}{2} \vert 001 \rangle
+ \frac{\alpha_1}{2} \vert 010 \rangle
+ \frac{\alpha_0}{2} \vert 011 \rangle
+ \frac{\alpha_0}{2} \vert 100 \rangle
- \frac{\alpha_1}{2} \vert 101 \rangle
- \frac{\alpha_1}{2} \vert 110 \rangle
+ \frac{\alpha_0}{2} \vert 111 \rangle
\end{aligned}
$$

When Alice measures her 2 qubits, there are four possible outcomes:
- Alice observes $\vert 00 \rangle$, the residual state is $\frac{\alpha_0}{2} \vert 0 \rangle + \frac{\alpha_1}{2} \vert 1 \rangle$
- Alice observes $\vert 01 \rangle$, the residual state is $\frac{\alpha_1}{2} \vert 0 \rangle + \frac{\alpha_0}{2} \vert 1 \rangle$
- Alice observes $\vert 10 \rangle$, the residual state is $\frac{\alpha_0}{2} \vert 0 \rangle - \frac{\alpha_1}{2} \vert 1 \rangle$
- Alice observes $\vert 11 \rangle$, the residual state is $-\frac{\alpha_1}{2} \vert 0 \rangle + \frac{\alpha_0}{2} \vert 1 \rangle$

Each outcomes has a probability of $\frac{1}{4}$. The residual state is the state of Bob's qubit. Alice can now transmit her measurement (in 2 classical bits) to Bob, and Bob an apply the appropriate unitary operator:
- If Alice's first classical bit is 1, then apply Pauli's $X$
- If Alice's second classical bit is 1, then apply Pauli's $Z$